# Setup 

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import subprocess
from datetime import date
import re
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('max_colwidth',500)
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
from importlib import reload


In [ ]:
# EDIT THESE VARIABLES
analysis_version = "2017_10_19"
project_dir = Path('/data/NNDSP') # needs to be pathlib.Path object
# project_dir = Path('/Users/rodgersleejg/data/hpc/NNDSP') # needs to be pathlib.Path object

In [ ]:
bids_dir = project_dir.joinpath('bids_2017_07_14')

# conf_script = mriqc_dir.joinpath('conf' + analysis_version + '.sh')

mriqc_dir  = project_dir.joinpath('anal/mriqc_files/other_files')
if not mriqc_dir.exists():
    mriqc_dir.mkdir()
output_folder =  project_dir / 'derivatives' / 'mriqc'
if not output_folder.exists():
    output_folder.mkdir()
classifier_output =  output_folder.joinpath('classifier')
if not classifier_output.exists():
    classifier_output.mkdir()
base_work_dir = output_folder.joinpath('work')
if not base_work_dir.exists():
    base_work_dir.mkdir()
log_dir = mriqc_dir.joinpath('swarm_output_' +  analysis_version)
if not log_dir.exists():
    log_dir.mkdir()
manual_qc = output_folder.joinpath('manual_qc_metrics.csv')
# swarm_path = mriqc_dir.joinpath('mriqc_' + analysis_version + '.cmd')

df_qc_full_pkl = Path('anal/mriqc_files/other_files/qc_pickle_for_v2_exploration.pklz')
mriqc_with_predictions = Path('derivatives/mriqc/with_mriqc_predictions.csv')
plottable_data = Path('derivatives/mriqc/classifer_plot_data.pklz')
plottable_data_euler = Path('derivatives/mriqc/classifer_plot_data_euler.pklz')

In [ ]:
%pwd
%cd {project_dir}
%pwd

In [ ]:
import anal.python_modules.inner_merge_and_report as pd_custom

# Mriqc and Euler number assessment

### Merge peformance sets

In [ ]:
df_roc_euler =  pd.read_pickle(plottable_data_euler)
df_roc_mriqc = pd.read_pickle(Path(plottable_data))
df_all_performance = df_roc_euler.merge(df_roc_mriqc,how = 'inner', on = ['MASKID','run','manual_qc_type'],suffixes = ('_euler','_mriqc'),indicator=True)
df_all_performance = df_all_performance.query('manual_qc_type == "MPRAGE"')

df_all_performance['manual_classification'] = df_all_performance.thresholded_euler.apply(lambda x:{0.0:'good',1.0:'bad'}[x])
df_all_performance.head()

# Good and bad scans

In [ ]:
df_bad = (df_all_performance.
 query('manual_classification == "bad"').
 sort_values(['prob_y_euler'],ascending = False))
df_bad

In [ ]:
df_good = (df_all_performance.
 query('manual_classification == "good"').
 sort_values(['prob_y_mriqc']).
          head(10))
df_good

#### chosen as good scan:

In [ ]:
df_all_performance.loc[(df_all_performance.MASKID == '1923') & (df_all_performance.run == 'run-001'),:]

In [ ]:
df_all_performance.loc[(df_all_performance.MASKID == '1923') & (df_all_performance.run == 'run-001'),:]

#### chosen as good scan:

In [ ]:
df_all_performance.loc[(df_all_performance.MASKID == '2018') & (df_all_performance.run == 'run-001'),:]

## Create snapshots dir

In [ ]:
mriqc_output = project_dir.joinpath('derivatives/mriqc')

In [ ]:
def make_symlink(snapshot_dir,scan_path,prefix = 'image'):
    
    from pathlib import Path
    if not snapshot_dir.exists():
        snapshot_dir.mkdir()
    s_path = Path(scan_path).absolute()    
    sym_path = prefix + '_'+ s_path.name
    !cd {snapshot_dir};ln -s {s_path} {sym_path}
    return snapshot_dir.joinpath(sym_path)
    
df_good_and_bad = pd.concat([df_good,df_bad],axis = 0)
df_good_and_bad = df_good_and_bad.query('manual_qc_type =="MPRAGE"')
df_good_and_bad = df_good_and_bad.assign(num = range(1, 1 + len(df_good_and_bad)))
qc_dir = output_folder.joinpath('good_bad_examplars_skullstripped')

In [ ]:
df_good_and_bad['glob_pat'] = df_good_and_bad.apply(lambda row :'work/sub-{m}_{r}/**/*T1w_conformed_corrected_skullstrip.nii.gz'.format(m = row.MASKID, r = row.run),axis = 1)
# pat = df_good_and_bad.loc[585,'glob_pat']
# print(pat)
# list(mriqc_output.glob(pat))
df_good_and_bad['skullstripped_path'] = (df_good_and_bad.
                                         apply(lambda row:
                                               list(mriqc_output.glob(row.glob_pat))[0],
                                               axis = 1)
                                        )

In [ ]:
qc_dir

### sanity check:

should all be ones:

In [ ]:
df_good_and_bad.groupby(['MASKID','run']).count()

In [ ]:
df_good_and_bad['sym_path'] = df_good_and_bad.apply(lambda row: make_symlink(qc_dir,row.skullstripped_path),axis = 1);

In [ ]:
df_good_and_bad.sym_path.values[0].stem.split('.')[0]

In [ ]:
qc_dir

In [ ]:

def make_montage(ulay=None,olay=None,image_dir=None,output_dir=None,cbar='FreeSurfer_Seg_i255',opacity='4',montx='3',monty='1',blowup='1',concatenate=False,prefix = None):
    
    ulay = Path(ulay)
    if not output_dir:
        output_dir = ulay.parent
        
    host = !hostname
    if 'nih' in host[0]:
        cmd = 'module load afni; '
    else:
        cmd = ""

        
    if not prefix:
        prefix = 'image'

    prefix += '_' + ulay.stem.split('.')[0]

    
    cmd += 'cd ' + ulay.parent.absolute().as_posix() + \
    '; \@chauffeur_afni' + \
    ' -ulay ' + ulay.as_posix()
    if olay:
        cmd += olay

    cmd += ' -prefix ' + prefix + \
    ' -cbar ' + cbar + \
    ' -opacity ' + opacity + \
    ' -do_clean' + \
    ' -montx ' + montx + \
    ' -monty ' + monty 
    
    if concatenate:
        cmd += ';echo Files for concatenation: ' + \
        ' $(ls {p}*png);'.format(p=prefix) + \
        'convert +append {p}*png {p}.jpg;'.format(p = prefix) + \
        'rm {p}.[a-z][a-z][a-z].png'.format(p = prefix)
        
    return cmd


for x,row in df_good_and_bad.iterrows():
    ulay = row.sym_path
    !{make_montage(ulay = ulay, montx ='1', prefix = row.manual_classification)}
    

# !ls {ulay.parent}


In [ ]:
snapshots_dir  =qc_dir.joinpath('snapshots')
if not snapshots_dir.exists():
    snapshots_dir.mkdir()
!mv {qc_dir}/*png {snapshots_dir.absolute()}

### write html

In [ ]:
output_html = qc_dir.joinpath('index.html')
summary_pics = list(qc_dir.glob('**/*png'))

summary_pics

In [ ]:
output_html.write_text("""
<HTML><TITLE>slicedir</TITLE><BODY BGCOLOR="#aaaaff">
""")
with output_html.open(mode='a') as f:
    for pic in summary_pics:
        f.write("""
            <a href="{pic}"><img src="{pic}" WIDTH=1000 > {pic_name}</a><br>
            """.format(pic='snapshots/' + pic.name ,pic_name =  pic.stem))
            
    f.write("""
</BODY></HTML>
""")

In [ ]:
<HTML><TITLE>slicedir</TITLE><BODY BGCOLOR="#aaaaff">

            <a href="snapshots/sub-102_ses-20120117.long.sub-102_template.jpg"><img src="snapshots/sub-102_ses-20120117.long.sub-102_template.jpg" WIDTH=1000 > sub-102_ses-20120117.long.sub-102_template</a><br>

            <a href="snapshots/sub-102_ses-20120120.long.sub-102_template.jpg"><img src="snapshots/sub-102_ses-20120120.long.sub-102_template.jpg" WIDTH=1000 > sub-102_ses-20120120.long.sub-102_template</a><br>

In [ ]:
print(output_html.read_text())

# Why the unpredicable bad scans?

Commands below should be run from the mriqc reports directory preferably locally

### possible human false positives

In [ ]:
df_fps = (df_euler_2.
 query('thresholded == 1.0').
 sort_values(['manual_qc_type','prob_y']).
 groupby('manual_qc_type').
 head())
df_fps

In [ ]:
'open sub-{' + ','.join(df_fps.MASKID) + '}*MP* # possible false positives'

+ 1024 has an odd distortion inside the cortex.
+ 0818 has ringing in the cortex

###  possible human false negatives

In [ ]:
df_fns = (df_euler_2.
 query('thresholded == 0.0').
 sort_values(['manual_qc_type','prob_y'],ascending = False).
 groupby('manual_qc_type').
 head())
df_fns

In [ ]:
'open sub-{' + ','.join(df_fns.MASKID) + '}*MP* # possible false negatives'